In [23]:
import itertools
import random

import simpy


random_seed = 42
gas_station_size = 200 #liters
threshold=40
fuel_tank_size = 50;
fuel_tank_level_range = [5, 25]
refuelling_speed = 2;
tank_truck_time = 300;
t_inter = [30, 300];
sim_time = 1000;

def car(name, env, gas_station, fuel_pump):
    
    fuel_tank_level = random.randint(*fuel_tank_level_range)
    print('%s arriving at gas station at %.1f' % (name, env.now))
    
    with gas_station.request() as req:
        start = env.now
        yield req
        
        liters_required  = fuel_tank_size -fuel_tank_level
        if fuel_pump.level < liters_required:
            print('calling truck due to car at %d' % env.now)
            yield env.process(tank_truck(env,fuel_pump))
        
        yield fuel_pump.get(liters_required)
            
        
        yield env.timeout(liters_required/refuelling_speed)
        
        print('%s finished refuelling %.1f liters in %.1f s' %(name, liters_required, env.now - start))

def gas_station_control (env, fuel_pump):
    while True:
        print('T: %d, Level: %1.f' % (env.now, fuel_pump.level))
        if fuel_pump.level / fuel_pump.capacity *100 < threshold:
            print('calling truck due to check at %d' % env.now)
            yield env.process(tank_truck(env,fuel_pump))
        
        yield env.timeout(10)
        

def tank_truck(env, fuel_pump):
    yield env.timeout(tank_truck_time)
    
    print('tank truck arriving at time %d' % env.now)
    ammount = fuel_pump.capacity - fuel_pump.level
    if ammount <= 0:
        print('Tank truck not needed')
        yield env.timeout(1)
    else:
        print('Tank truck refuelling %.1f liters.' % ammount)
        yield fuel_pump.put(ammount)
    
def car_generator(env, gas_station, fuel_pump):
    for i in itertools.count():
        yield env.timeout(random.randint(*t_inter))
        
        env.process(car('Car %d' %i, env, gas_station, fuel_pump))
        
print('Gas Station Refuelling Sim')

#random.seed(random_seed)

#create env:
env = simpy.Environment()
gas_station = simpy.Resource(env,2)
fuel_pump = simpy.Container(env, gas_station_size, init=gas_station_size)

env.process(gas_station_control(env,fuel_pump))
env.process(car_generator(env,gas_station, fuel_pump))


env.run(until=sim_time)

Gas Station Refuelling Sim
T: 0, Level: 200
T: 10, Level: 200
T: 20, Level: 200
T: 30, Level: 200
T: 40, Level: 200
T: 50, Level: 200
T: 60, Level: 200
T: 70, Level: 200
T: 80, Level: 200
T: 90, Level: 200
T: 100, Level: 200
T: 110, Level: 200
Car 0 arriving at gas station at 117.0
T: 120, Level: 160
T: 130, Level: 160
Car 0 finished refuelling 40.0 liters in 20.0 s
T: 140, Level: 160
T: 150, Level: 160
T: 160, Level: 160
T: 170, Level: 160
T: 180, Level: 160
T: 190, Level: 160
T: 200, Level: 160
T: 210, Level: 160
T: 220, Level: 160
T: 230, Level: 160
T: 240, Level: 160
T: 250, Level: 160
T: 260, Level: 160
T: 270, Level: 160
Car 1 arriving at gas station at 272.0
T: 280, Level: 127
Car 1 finished refuelling 33.0 liters in 16.5 s
T: 290, Level: 127
T: 300, Level: 127
T: 310, Level: 127
T: 320, Level: 127
T: 330, Level: 127
T: 340, Level: 127
T: 350, Level: 127
T: 360, Level: 127
T: 370, Level: 127
T: 380, Level: 127
T: 390, Level: 127
T: 400, Level: 127
T: 410, Level: 127
T: 420, Leve

In [19]:
mygenerator = (x for x in range(3))
for i in mygenerator:
    print(i)

0
1
2
